In [1]:
import random
import pandas as pd 
import os
        
### Create file csv data from sample
data_source = 'website_classification.csv'
folder = pd.read_csv(data_source)

In [3]:
folder['Category']

0       Travel
1       Travel
2       Travel
3       Travel
4       Travel
         ...  
1403     Adult
1404     Adult
1405     Adult
1406     Adult
1407     Adult
Name: Category, Length: 1408, dtype: object